# NLP HSE School

In [5]:
from pymystem3 import Mystem
import pandas as pd
import re

Перенесем словарь РуСентиЛекс в DataFrame.

In [6]:
with open('RuSentiLex2017_revised_2utf.txt', 'r') as f:
    dict_rsl = f.readlines()
dict_rsl = pd.DataFrame(
    [string.split(',') for string in dict_rsl[18:]],
    columns=['word', 'part_of_speech', 'lemma', 'sentiment', 'source', 'ambiguity', 'smth1', 'smth2']
)

In [7]:
def sentiment_calc(obj):
    obj = obj.replace(' ', '')
    if obj == 'positive':
        return 1
    elif obj == 'negative':
        return -1
    else:
        return 0

In [8]:
dict_rsl['sentiment'] = dict_rsl['sentiment'].apply(sentiment_calc)

In [9]:
dict_compar = dict_rsl[['lemma', 'sentiment']].copy()

In [10]:
vals = dict_compar['lemma'].values.tolist()
dict_compar_ = dict_compar[['sentiment']].copy()
dict_compar_.index = dict_compar['lemma']
dict_compar_map = dict_compar_.to_dict()['sentiment']

Собираем нужные файлы

In [11]:
# Функция для чтения набора файлов относящихся к n статье
def art_read(num):
    with open('Texts/art{}.txt'.format(num), 'r', encoding='utf-8') as f:
        text_lines = f.readlines()
        f.seek(0)
        text_full = f.read()

    with open('Texts/art{}.opin.txt'.format(num), 'r', encoding='utf-8') as f:
        opin_lines = f.readlines()
        f.seek(0)
        opin_full = f.read()

    with open('Texts/art{}.ann'.format(num), 'r') as f:
        entities_lines = f.readlines()
        f.seek(0)
        entities_full = f.read()
    return {'txt_l':text_lines, 'txt_f':text_full, 'opin_l':opin_lines,
            'opin_f':opin_full, 'ent_l':entities_lines, 'ent_f':entities_full}

In [12]:
art = art_read(1)
text_lines = art['txt_l']
text_full = art['txt_f']
ent_lines = art['ent_l']
ent_full = art['ent_f']

Рассмотрение сущностей

In [24]:
def entity_transform(lemmas):
    result = ''.join(lemmas)
    result = result.replace('\n', '')
    return result

m = Mystem()

def entity_pairs(ent_lines):
    entities = pd.DataFrame(
        [
            sum([re.findall(r'T[0-9]+\t([A-Z]+)?', string) for string in ent_lines], []),
            sum([re.findall(r'[0-9]+ [0-9]+\t(.+?)\n', string) for string in ent_lines], [])
        ], index = ['type', 'name']
    ).T
    entities = entities[(entities['name']!='Author') & (entities['name']!='Unknown')].dropna().reset_index(drop=True)
    entities['name'] = entities['name'].apply(m.lemmatize)
    entities['name'] = entities['name'].apply(entity_transform)
    entities_unique = entities.drop_duplicates()
    entities_unique_list = entities_unique['name'].values.tolist()
    list_of_pairs = [(entities_unique_list[p1], entities_unique_list[p2])
                     for p1 in range(len(entities_unique_list))
                     for p2 in range(p1+1,len(entities_unique_list))]
    return list_of_pairs

list_of_pairs = entity_pairs(ent_lines)

Анализ первого текста

In [25]:
def sentance_extraction(text, pairs):
    lemmas = m.lemmatize(text)
    sentence = ''.join(lemmas)
    sentence_ent = []
    for p in pairs:
        if (p[0] in sentence) & (p[1] in sentence):
            first_char = sentence.find(p[0])
            second_char = sentence.find(p[1])
            sorted_ent = sorted(
                [
                    (first_char,p[0]), (second_char, p[1])
                ],
                key = lambda element : element[0]
            )

            if sorted_ent not in sentence_ent:
                sentence_ent.append(sorted_ent)

    result = []
    for p in sentence_ent:
        if p[1][0] - p[0][0] > 10:

            words = []
            for v in vals:
                if v in sentence[p[0][0]:p[1][0]]:
                    if v not in words:
                        words.append(v)

            result.append(
                [p[0][1],
                p[1][1],
                sum(list(map(lambda x: dict_compar_map[x], words))),
                text]
            )
    return result

In [28]:
opinions = []
for i in range(len(text_lines)):
    opinions += sentance_extraction(text_lines[i].replace("{Author, Unknown} ",""), list_of_pairs)

In [29]:
pd.DataFrame(opinions)

,0,1,2,3
0,сми,нато,-1,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
1,сми,россия,-2,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
2,иран,нато,-1,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
3,иран,россия,-2,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
4,финляндия,нато,-1,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
5,финляндия,россия,-2,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
6,нато,россия,-1,СМИ Ирана: Финляндия не хочет вступать в НАТО ...
7,сирия,румийлан,0,Зачем американцам понадобилось перебросить 250...
8,сирия,Khorasan,0,Зачем американцам понадобилось перебросить 250...
9,америка,сирия,0,Зачем американцам понадобилось перебросить 250...
